# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [13]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


In [6]:
df.size

14

## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [75]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature= 0.7)


In [76]:
prompt = ChatPromptTemplate.from_template(
    "Please provide a detailed description of a {product}. Include key features and benefits of the review."
)

In [77]:

chain = LLMChain(llm=llm, prompt=prompt)

In [78]:
product = "Waterproof Phone Pouch"  # Selecting a product from the available data
chain.run(product)

'A Waterproof Phone Pouch is a must-have accessory for anyone who wants to protect their phone from water damage while engaging in water-related activities such as swimming, snorkeling, kayaking, or simply relaxing at the beach or by the pool.\n\nKey Features:\n\n1. Waterproof Material: The pouch is typically made of high-quality, durable PVC or TPU material that is completely waterproof, ensuring that your phone remains dry and functional even when submerged in water.\n\n2. Secure Seal: The pouch features a secure seal closure mechanism, such as a zip-lock or snap-lock system, that effectively keeps water out and prevents any leakage.\n\n3. Touchscreen Compatibility: The pouch is designed to allow full touchscreen functionality, so you can easily use your phone to take photos, answer calls, or send messages without having to remove it from the pouch.\n\n4. Universal Fit: Most waterproof phone pouches are designed to fit a wide range of phone models, including iPhones, Samsung Galaxy p

## SimpleSequentialChain

In [15]:
from langchain.chains import SimpleSequentialChain

In [16]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
    "Please provide a concise description of a {product}. Include key features and benefits."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [17]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the following product description, provide a rating out of 5 stars and explain why: {input}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [19]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A waterproof phone pouch is a protective case that safeguards your phone from water damage while allowing you to use it in various water-related activities. Key features include a secure seal to keep out water, clear touch-sensitive material for easy phone operation, and a lanyard or strap for added convenience. Benefits include peace of mind knowing your phone is protected, ability to take photos and videos underwater, and versatility for use during beach trips, water sports, or rainy weather.
I would give this product a 5-star rating. The waterproof phone pouch offers excellent protection for your phone from water damage, which is crucial for anyone who enjoys water-related activities. The secure seal ensures your phone stays dry, while the touch-sensitive material allows for easy operation. The added convenience of a lanyard or strap makes it easy to carry around. The ability to take photos and videos underwater is a great feature, and

'I would give this product a 5-star rating. The waterproof phone pouch offers excellent protection for your phone from water damage, which is crucial for anyone who enjoys water-related activities. The secure seal ensures your phone stays dry, while the touch-sensitive material allows for easy operation. The added convenience of a lanyard or strap makes it easy to carry around. The ability to take photos and videos underwater is a great feature, and the versatility of being able to use it during various water activities or even in rainy weather is a huge plus. Overall, this product provides peace of mind and great functionality, making it well worth a 5-star rating.'

**Repeat the above twice for different products**

## Example 1

In [21]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Please provide a concise description of a {product}. Include key features and benefits."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the following product description, provide a rating out of 5 stars and explain why: {input}"
)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
product = 'Queen Size Sheet Set'
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A Queen Size Sheet Set includes a fitted sheet, flat sheet, and two standard pillowcases designed to fit a queen size bed. Key features include a soft and durable fabric, deep pockets for a secure fit on mattresses up to 18 inches thick, and a variety of colors and patterns to match any decor. The benefits of a Queen Size Sheet Set include easy care, machine washable fabric, and a comfortable and luxurious feel for a good night's sleep.
I would give this Queen Size Sheet Set a 5-star rating. It offers a great combination of practical and luxurious features. The soft and durable fabric, deep pockets for a secure fit, and variety of colors and patterns make it versatile and long-lasting. The easy care, machine washable fabric is a huge benefit for busy individuals looking for low maintenance bedding. Overall, this sheet set seems to provide everything one would need for a comfortable and restful night's sleep.

> Finished chain.


"I would give this Queen Size Sheet Set a 5-star rating. It offers a great combination of practical and luxurious features. The soft and durable fabric, deep pockets for a secure fit, and variety of colors and patterns make it versatile and long-lasting. The easy care, machine washable fabric is a huge benefit for busy individuals looking for low maintenance bedding. Overall, this sheet set seems to provide everything one would need for a comfortable and restful night's sleep."

## Example 2

In [37]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)  # Using 0.7 for a balance between creativity and consistency

# First translate any non-English text
first_prompt = ChatPromptTemplate.from_template(
    """Translate the following product name and any associated text to English if it's not already in English: {product}
    If it's already in English, return it unchanged."""
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="english_product"
                    )

# Then analyze the English product text
second_prompt = ChatPromptTemplate.from_template(
    """Analyze this product: {english_product}
    Provide a consice description including key features and target audience."""
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="product_analysis"
                    )

third_prompt = ChatPromptTemplate.from_template(
    "Summarize the key selling points of this product in 1-2 sentences: {product_analysis}"
)

chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="summary"
                      )

fourth_prompt = ChatPromptTemplate.from_template(
    """Based on this product analysis: {product_analysis}
    Translate the product description to Spanish to reach more customers"""
)

chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="spanish_description"
                     )

fifth_prompt = ChatPromptTemplate.from_template(
    """Based on this product summary: {summary}
    Write a compelling marketing message that highlights the main benefits."""
)

chain_five = LLMChain(llm=llm, prompt=fifth_prompt,
                      output_key="marketing_message"
                     )

sixth_prompt = ChatPromptTemplate.from_template(
    """Based on the product analysis: {product_analysis}
    Provide a competitive analysis and suggested retail price range."""
)

chain_six = LLMChain(llm=llm, prompt=sixth_prompt,
                     output_key="pricing_analysis"
                    )

seventh_prompt = ChatPromptTemplate.from_template(
    """Based on the product analysis: {product_analysis}
    Rate this product on a scale of 1-5 stars and provide a detailed justification for the rating.
    Consider factors like quality, features, value for money, and target audience satisfaction."""
)

chain_seven = LLMChain(llm=llm, prompt=seventh_prompt,
                      output_key="product_rating"
                     )

# Set up the overall chain
overall_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four, chain_five, chain_six, chain_seven],
    verbose=True
)

product = "L'Or Espresso Café"  # Using a product from the dataset
overall_chain.run(product)




> Entering new SimpleSequentialChain chain...
L'Or Espresso Café --> L'Or Espresso Café
L'Or Espresso Café is a premium coffee brand that offers a range of high-quality espresso blends made from carefully selected beans. The key features of this product include its rich and intense flavors, smooth crema, and convenient Nespresso-compatible capsules for easy brewing.

The target audience for L'Or Espresso Café is coffee enthusiasts who appreciate a luxurious coffee experience at home or in the office. This product is ideal for those who value convenience but do not want to compromise on taste and quality. With its elegant packaging and sophisticated flavor profiles, L'Or Espresso Café is perfect for those looking to indulge in a gourmet coffee experience.


L'Or Espresso Café offers premium espresso blends with rich flavors, smooth crema, and convenient Nespresso-compatible capsules for easy brewing, making it an ideal choice for coffee enthusiasts who value luxury and convenience wit

"I would rate L'Or Espresso Café capsules a 4 out of 5 stars. \n\nQuality: L'Or Espresso Café capsules are known for their high-quality coffee blends that offer intense flavors and smooth crema. The brand focuses on providing a luxurious coffee experience, which sets them apart from their competitors. \n\nFeatures: The Nespresso-compatible capsules are convenient and easy to use, making it simple for consumers to enjoy a premium cup of coffee at home. The variety of blends available also caters to different taste preferences, giving customers options to choose from. \n\nValue for Money: While L'Or Espresso Café capsules are positioned as a premium brand, the suggested retail price range of $5.99 to $7.99 for a pack of 10 capsules is reasonable for the quality of coffee they provide. Customers looking for a high-quality coffee experience may find the price worth it.\n\nTarget Audience Satisfaction: L'Or Espresso Café caters to consumers who appreciate premium coffee and are willing to p

## SequentialChain

In [44]:
from langchain.chains import SequentialChain

In [45]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    """You are a professional translator.
    Please translate this review to English if it's not already in English.
    If it's already in English, return it unchanged.
    
    Review: {review}"""
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="english_review" #Give a name to your output
                    )


In [46]:
second_prompt = ChatPromptTemplate.from_template(
    """You are a professional review analyst.
    Please provide a concise summary of this review in 1-2 sentences.
    
    Review: {english_review}"""
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="review_summary"
                    )


In [47]:
# prompt template 3: translate summary back to original language
third_prompt = ChatPromptTemplate.from_template(
    """You are a professional translator.
    Please translate this review summary to the original language of the review.
    If the original review was in English, return it unchanged.
    
    Review Summary: {review_summary}"""
)
# chain 3: input = review_summary, output = translated_summary 
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_summary"
                      )


In [48]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    """You are a helpful customer service agent.
    Based on the original review and its summary, write a brief follow-up message to the customer.
    Thank them for their review and address their main points.
    
    Original Review: {english_review}
    Review Summary: {review_summary}"""
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [49]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "review_summary", "followup_message"],
    verbose=True
)

In [50]:
review = df.Review[5]
print(review)

overall_chain(review)

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !?


> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'english_review': "Review: I find the taste mediocre. The foam doesn't last, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'review_summary': "The reviewer is dissatisfied with the taste of the product, noting that the foam doesn't last and suspects it may be an old batch or counterfeit compared to similar products purchased in stores.",
 'followup_message': 'Dear valued customer,\n\nThank you for taking the time to leave a review. We apologize that you found the taste of our product to be mediocre and that the foam did not last as expected. We can assure you that our products are not old batches or counterfeit, but we would like to investigate further to ensure the quality of our products meets our standards. Please reach out to our customer service tea

**Repeat the above twice for different products or reviews**

## Example 1

In [51]:
review = df.Review[6]
print(review)


overall_chain(review)

Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.


> Entering new SequentialChain chain...

> Finished chain.


{'review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'english_review': "Review: The heater is very pretty and heats up very quickly, it is very functional, just need to see how long it lasts, I've only been using it for 3 days.",
 'review_summary': 'The reviewer finds the heater aesthetically pleasing and effective in heating up quickly, but is uncertain about its longevity after only 3 days of use.',
 'followup_message': 'Dear valued customer,\n\nThank you for taking the time to leave a review of our heater. We are delighted to hear that you find it aesthetically pleasing and that it heats up quickly. We understand your concern about its longevity after only 3 days of use. Rest assured, our products are designed to be durable and long-lasting. If you have any issues or questions in the future, please do not hesitate to reach out to us.\n\nThank you for choosing our heater!\n\nWarm regards,\n[Your Name]  \

## Example 2

In [52]:
review = df.Review[4]
print(review)


overall_chain(review)

 I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.


> Entering new SequentialChain chain...

> Finished chain.


{'review': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'english_review': "Review: I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'review_summary': 'The reviewer enjoyed the product initially but found that it stopped working after a few months, eve

## Router Chain

In [53]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [54]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [55]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [56]:
llm = ChatOpenAI(temperature=0)

In [58]:
# This loop creates a dictionary of LLMChains for different subject areas (physics, math, history, etc)
# Each chain is configured with a specific prompt template for that subject
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]  # Get subject name (e.g. "physics", "math")
    prompt_template = p_info["prompt_template"]  # Get the prompt template for this subject
    prompt = ChatPromptTemplate.from_template(template=prompt_template)  # Create prompt from template
    chain = LLMChain(llm=llm, prompt=prompt)  # Create chain with prompt and language model
    destination_chains[name] = chain  # Add chain to dictionary with subject name as key
    
# Create formatted strings describing each subject chain for the router template
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)  # Join descriptions with newlines

In [71]:
destinations

['physics: Good for answering questions about physics',
 'math: Good for answering math questions',
 'History: Good for answering history questions',
 'computer science: Good for answering computer science questions']

The dictionary approach is used here for the Router Chain pattern, which acts like a "traffic director" for different types of questions. Here's why this approach is useful:
Organization and Access:
The dictionary structure allows quick lookup of the appropriate chain based on the subject
Instead of having multiple if/else statements, we can directly access the right chain using destination_chains["physics"] or destination_chains["math"]

In [59]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [62]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [63]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [64]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\edwar\AppData\Local\Temp\ipykernel_35520\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [65]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum of wavelengths and intensities that depend only on the temperature of the body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body is a function of its temperature and the wavelength of the radiation. Black body radiation plays a key role in understanding the behavior of objects at high temperatures and in the field of thermal radiation."

In [66]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [67]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [68]:
# Example 1: Testing different academic questions
academic_questions = [
    "How does quantum entanglement work?",  # Physics
    "Explain the concept of derivatives in calculus",  # Math
    "What were the main causes of World War I?",  # History
    "How does a neural network learn?",  # Computer Science
]

print("Academic Questions Test:")
print("-" * 50)
for question in academic_questions:
    print(f"\nQuestion: {question}")
    print(chain.run(question))
    print("-" * 50)

Academic Questions Test:
--------------------------------------------------

Question: How does quantum entanglement work?


> Entering new MultiPromptChain chain...
physics: {'input': 'How does quantum entanglement work?'}
> Finished chain.
Quantum entanglement is a phenomenon where two or more particles become connected in such a way that the state of one particle is directly related to the state of the other, regardless of the distance between them. This means that measuring the state of one particle instantly determines the state of the other, even if they are light-years apart.

The exact mechanism behind quantum entanglement is still not fully understood, but it is believed to involve a non-local connection between the particles that allows them to share information instantaneously. This phenomenon is a key aspect of quantum mechanics and has been experimentally verified through various tests and experiments.

In summary, quantum entanglement is a mysterious and fascinating aspec

In [72]:
# Example 2: Testing interdisciplinary questions
interdisciplinary_questions = [
    "How is calculus used in physics to describe motion?",  # Math + Physics
    "How did the invention of computers impact historical research?",  # Computer Science + History
    "How are algorithms used to analyze historical data?",  # Computer Science + History
    "What mathematical principles are used in modern cryptography?",  # Math + Computer Science
]

print("\nInterdisciplinary Questions Test:")
print("-" * 50)
for question in interdisciplinary_questions:
    print(f"\nQuestion: {question}")
    print(chain.run(question))
    print("-" * 50)




Interdisciplinary Questions Test:
--------------------------------------------------

Question: How is calculus used in physics to describe motion?


> Entering new MultiPromptChain chain...
physics: {'input': 'How is calculus used in physics to describe motion?'}
> Finished chain.
Calculus is used in physics to describe motion by analyzing the relationships between position, velocity, and acceleration. By taking derivatives and integrals of these quantities with respect to time, we can determine how an object's motion changes over time. For example, the derivative of position gives us velocity, and the derivative of velocity gives us acceleration. Integrating acceleration gives us velocity, and integrating velocity gives us position. This allows us to accurately describe and predict the motion of objects in the physical world.
--------------------------------------------------

Question: How did the invention of computers impact historical research?


> Entering new MultiPromptChain 

In [74]:
# Example 3: Testing real-world application questions
application_questions = [
    "How do self-driving cars use physics principles?",  # Physics
    "How is probability used in weather forecasting?",  # Math
    "How did the development of the internet change modern warfare?",  # History
    "How does blockchain technology work?",  # Computer Science
    "What is cancer?"
]

print("\nReal-world Application Questions Test:")
print("-" * 50)
for question in application_questions:
    print(f"\nQuestion: {question}")
    print(chain.run(question))
    print("-" * 50)


Real-world Application Questions Test:
--------------------------------------------------

Question: How do self-driving cars use physics principles?


> Entering new MultiPromptChain chain...
physics: {'input': 'How do self-driving cars use physics principles?'}
> Finished chain.
Self-driving cars use physics principles in a variety of ways to navigate and operate safely. Some key ways include:

1. Sensors and cameras on the car detect the car's surroundings and use principles of optics and electromagnetic waves to interpret and react to the environment.

2. The car's computer system uses principles of mechanics and kinematics to calculate the best path and speed to reach its destination while avoiding obstacles.

3. The car's control systems use principles of dynamics and control theory to adjust steering, acceleration, and braking to maintain stability and control.

Overall, self-driving cars rely on a combination of physics principles to operate autonomously and safely on the road